In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Idiosyncratic Volatility Strategy (IV)

### Part a)

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Import the data, specifying the index column if applicable
data = pd.read_csv("/Users/eperazz2/Desktop/EPFL/Cours Semestre 2/Investments/Project/Data/data.csv", index_col=0)

# Display the first few rows to verify
data.sort_values(by=['permno','date'],inplace=True)
data.groupby('date').first().sort_values(by='date')
print(data.head())

    permno        date       ret  shrout      prc  siccd     tmytm    vwretd
94   10001  2009-12-31  0.162621  4361.0  10.3000   4925  0.000032  0.028475
93   10001  2010-01-29 -0.018932  4361.0  10.0600   4925  0.000013 -0.037172
92   10001  2010-02-26 -0.000656  4361.0  10.0084   4925  0.000061  0.034744
91   10001  2010-03-31  0.020643  4361.0  10.1700   4925  0.000112  0.063668
90   10001  2010-04-30  0.124385  6070.0  11.3900   4925  0.000118  0.020036


#### Calculate excess returns

In [6]:
# Calculate excess returns:
# ['tmytm'] is the column representing the risk-free rate
data['excess_ret'] = data['ret'] - data['tmytm']
data['excess_market_ret'] = data['vwretd'] - data['tmytm']

##### Regression for IV

In [7]:
# Drop nan values
data = data.dropna().copy()

In [8]:
# Filter stocks with at least 36 months of data
data['num_months'] = data.groupby('permno')['date'].transform('count')
data = data[data['num_months'] >= 36].copy()

# Rolling window size
window = 5 * 12  # 5 years * 12 months

# Calculate idiosyncratic volatility for each stock using a rolling window approach
def calculate_idiosyncratic_volatility(group):
    if len(group) < window:
        return pd.Series([np.nan] * len(group), index=group.index)
    
    residuals = pd.Series(index=group.index)
    
    # iterate over each stock (permno group), sliding a window of 5 years (60 months) across the data
    for i in range(len(group) - window + 1):
        window_data = group.iloc[i:i+window]
        excess_returns_window = window_data['excess_ret']
        market_excess_returns_window = sm.add_constant(window_data['excess_market_ret'])
        
        # Regress excess_ret against excess_market_ret with OLS to estimate the residuals' standard deviation 
        model = sm.OLS(excess_returns_window, market_excess_returns_window)
        results = model.fit()
        
        residuals.iloc[i + window - 1] = np.std(results.resid)
    
    return residuals

# Apply the function to each stock
data['idiosyncratic_volatility'] = data.groupby('permno').apply(calculate_idiosyncratic_volatility).reset_index(level=0, drop=True)

# Remove rows where idiosyncratic volatility could not be computed
data = data.dropna(subset=['idiosyncratic_volatility']).copy()

# Output the data with the new idiosyncratic volatility column
print(data[['permno', 'date', 'idiosyncratic_volatility']])

# Save 
data.to_csv("/Users/eperazz2/Desktop/data_with_volatility.csv", index=False)

/var/folders/mm/gtq67xxs28b__g9m9949jrm00000gn/T/ipykernel_49820/1875172710.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data['idiosyncratic_volatility'] = data.groupby('permno').apply(calculate_idiosyncratic_volatility).reset_index(level=0, drop=True)


         permno        date  idiosyncratic_volatility
127       10001  2014-11-28                  0.063269
126       10001  2014-12-31                  0.060464
125       10001  2015-01-30                  0.062201
124       10001  2015-02-27                  0.062240
123       10001  2015-03-31                  0.062239
...         ...         ...                       ...
1438321   93426  2023-08-31                  0.061566
1438320   93426  2023-09-29                  0.059316
1438319   93426  2023-10-31                  0.059753
1438318   93426  2023-11-30                  0.060708
1438317   93426  2023-12-29                  0.061170

[980533 rows x 3 columns]


##### Winsorize at 5th and 95th percentiles

In [9]:
# Apply the function to each stock
data['idiosyncratic_volatility'] = data.groupby('permno').apply(calculate_idiosyncratic_volatility).reset_index(level=0, drop=True)

# Remove rows where idiosyncratic volatility could not be computed
data = data.dropna(subset=['idiosyncratic_volatility']).copy()

# Winsorize the idiosyncratic volatility
idiosyncratic_volatility = data['idiosyncratic_volatility'].values
idio_vol_5th_percentile = np.percentile(idiosyncratic_volatility, 5)
idio_vol_95th_percentile = np.percentile(idiosyncratic_volatility, 95)
idiosyncratic_volatility_winsorized = np.clip(idiosyncratic_volatility, idio_vol_5th_percentile, idio_vol_95th_percentile)

# Assign the winsorized idiosyncratic volatility back to the DataFrame
data['idiosyncratic_volatility'] = idiosyncratic_volatility_winsorized

# Output the DataFrame with idiosyncratic volatility estimates
print(data[['permno', 'date', 'idiosyncratic_volatility']])

# Save the output data if needed
data.to_csv("/Users/eperazz2/Desktop/data_with_volatility.csv", index=False)

/var/folders/mm/gtq67xxs28b__g9m9949jrm00000gn/T/ipykernel_49820/2280716005.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data['idiosyncratic_volatility'] = data.groupby('permno').apply(calculate_idiosyncratic_volatility).reset_index(level=0, drop=True)


         permno        date  idiosyncratic_volatility
353       10006  1973-12-31                  0.080622
352       10006  1974-01-31                  0.080504
351       10006  1974-02-28                  0.080328
350       10006  1974-03-29                  0.080332
349       10006  1974-04-30                  0.077265
...         ...         ...                       ...
1438321   93426  2023-08-31                  0.061566
1438320   93426  2023-09-29                  0.059316
1438319   93426  2023-10-31                  0.059753
1438318   93426  2023-11-30                  0.060708
1438317   93426  2023-12-29                  0.061170

[687609 rows x 3 columns]


### Part b)